In [11]:
import pandas as pd 
import zipfile
import os 

Extracting column names from GDELT documentation 

In [12]:
headers_url = "https://www.gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt"
colnames = pd.read_csv(headers_url, sep="\t").columns.tolist()
print(colnames)

['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_FullName', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 'ActionGeo_Lat', 

Casting Types early on to avoid warnings and low memory
IsRootEvent is casted as str to avoid problems of with numbers starting with 0 its original type is either (logical , binary or byte)

In [22]:
dataTypes = {0:int,1:int,2:int,3:int,4:float,5:str,6:str,7:str,8:str,9:str,10:str,11:str,
             12:str,13:str,14:str,15:str,16:str,17:str,18:str,19:str,20:str,21:str,22:str,
             23:str,24:str,25:str,26:str,27:str,28:str,29:int,30:float,31:int,32:int,33:int,
             34:float,35:int,36:str,37:str,38:str,39:float,40:float,41:int,42:int,43:str,44:str,
             45:str,46:float,47:float,48:int,49:int,50:str,51:str,52:str,53:float,54:float,
             55:int,56:int,57:str}

Parsing all files

In [13]:
raw_dir = r"C:\Users\bahak\Gdelt project\data\raw"
clean_dir = r"C:\Users\bahak\Gdelt project\data\clean"
os.makedirs(clean_dir, exist_ok=True)

for filename in os.listdir(raw_dir):
    clean_file_path = os.path.join(clean_dir, f"{filename.split('.')[0]}.csv")
    if not os.path.exists(clean_file_path):
       print(f"processing {filename}...")
       if filename.endswith(".zip"):
           zip_path = os.path.join(raw_dir, filename)
           date_part = filename.split(".")[0]
           with zipfile.ZipFile(zip_path, "r") as z:
            csv_name = z.namelist()[0]
            with z.open(csv_name) as f:
                df = pd.read_csv(f, sep="\t", names=colnames, header=None,low_memory=False)
                output_path = os.path.join(clean_dir, f"{date_part}.csv")
                df.to_csv(output_path, index=False)
    else:
       print(f"File {clean_file_path} already exists, skipping...")
print("All files loaded and saved data/clean")

processing 20220601.export.CSV.zip...
processing 20220602.export.CSV.zip...
processing 20220603.export.CSV.zip...
processing 20220604.export.CSV.zip...
processing 20220605.export.CSV.zip...
processing 20220606.export.CSV.zip...
processing 20220607.export.CSV.zip...
processing 20220608.export.CSV.zip...
processing 20220609.export.CSV.zip...
processing 20220610.export.CSV.zip...
processing 20220611.export.CSV.zip...
processing 20220612.export.CSV.zip...
processing 20220613.export.CSV.zip...
processing 20220614.export.CSV.zip...
processing 20220615.export.CSV.zip...
processing 20220616.export.CSV.zip...
processing 20220617.export.CSV.zip...
processing 20220618.export.CSV.zip...
processing 20220619.export.CSV.zip...
processing 20220620.export.CSV.zip...
processing 20220621.export.CSV.zip...
processing 20220622.export.CSV.zip...
processing 20220623.export.CSV.zip...
processing 20220624.export.CSV.zip...
processing 20220625.export.CSV.zip...
processing 20220626.export.CSV.zip...
processing 2

In [15]:
from pandasgui import show
df = pd.read_csv(r'C:\Users\bahak\Gdelt project\data\clean\20230401.csv')
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\bahak\AppData\Local\Temp\ipykernel_5832\3488340625.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\bahak\AppData\Local\Temp\ipykernel_5832\3488340625.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\bahak\AppData\Local\Temp\ipykernel_5832\3488340625.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\bahak\AppData\Local\Temp\ipykernel_5832\

Filtering relevent rows from total datasets

In [14]:
import os
import pandas as pd

clean_dir = r"C:\Users\bahak\Gdelt project\data\clean"
output_path = r"C:\Users\bahak\Gdelt project\data\PALISR\filtered.csv"
os.makedirs(r"C:\Users\bahak\Gdelt project\data\PALISR", exist_ok=True)
keywords = r'PES|PAL|ISR'

first_write = True  # To control whether to write header
lastfile = ''
for filename in os.listdir(clean_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(clean_dir, filename)
        print(f"Processing {filename}")
        lastfile = filename
        # Load file with original types
        df = pd.read_csv(file_path, low_memory=False)

        # Only convert columns 5 and 15 to string for filtering
        col5 = df.iloc[:, 5].astype(str)
        col15 = df.iloc[:, 15].astype(str)

        # Apply filtering
        mask = col5.str.contains(keywords, na=False) | col15.str.contains(keywords, na=False)
        filtered = df[mask]

        # Append filtered rows to output CSV
        filtered.to_csv(output_path, mode='a', index=False, header=first_write)
        first_write = False  # Only write header for first file
print(f"lastfile processed : {lastfile}")

Processing 20220601.csv
Processing 20220602.csv
Processing 20220603.csv
Processing 20220604.csv
Processing 20220605.csv
Processing 20220606.csv
Processing 20220607.csv
Processing 20220608.csv
Processing 20220609.csv
Processing 20220610.csv
Processing 20220611.csv
Processing 20220612.csv
Processing 20220613.csv
Processing 20220614.csv
Processing 20220615.csv
Processing 20220616.csv
Processing 20220617.csv
Processing 20220618.csv
Processing 20220619.csv
Processing 20220620.csv
Processing 20220621.csv
Processing 20220622.csv
Processing 20220623.csv
Processing 20220624.csv
Processing 20220625.csv
Processing 20220626.csv
Processing 20220627.csv
Processing 20220628.csv
Processing 20220629.csv
Processing 20220630.csv
Processing 20220701.csv
Processing 20220702.csv
Processing 20220703.csv
Processing 20220704.csv
Processing 20220705.csv
Processing 20220706.csv
Processing 20220707.csv
Processing 20220708.csv
Processing 20220709.csv
Processing 20220710.csv
Processing 20220711.csv
Processing 20220